<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/03_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 3: Classificaiton

Setelah membahas regresi di bab sebelumnya, bab ini berfokus pada tugas **klasifikasi**. Dataset utama yang digunakan adalah MNIST, yang terdiri dari 70.000 gambar kecil angka tulisan tangan.

* **Pengenalan Klasifikasi**: Tugas untuk memprediksi kelas atau kategori dari sebuah instance. Bab ini memulai dengan kasus sederhana, yaitu **klasifikasi biner** (dua kelas), sebelum beralih ke **klasifikasi multikelas**.

* **Metrik Kinerja untuk Klasifikasi**: Bab ini menekankan bahwa akurasi saja tidak cukup untuk mengevaluasi model klasifikasi, terutama pada dataset yang tidak seimbang (*skewed dataset*). Metrik yang lebih baik diperkenalkan:
    * ***Confusion Matrix***: Menunjukkan jumlah prediksi yang benar dan salah untuk setiap kelas. Istilah kunci: *True Positive* (TP), *True Negative* (TN), *False Positive* (FP), *False Negative* (FN).
    * ***Precision***: Akurasi dari prediksi positif (`TP / (TP + FP)`). Menjawab pertanyaan: "Dari semua yang diprediksi sebagai kelas X, berapa persen yang benar?"
    * ***Recall*** (atau *Sensitivity*): Rasio instance positif yang berhasil dideteksi oleh model (`TP / (TP + FN)`). Menjawab pertanyaan: "Dari semua instance kelas X yang sebenarnya, berapa persen yang berhasil terdeteksi?"
    * ***F1 Score***: Rata-rata harmonik dari *precision* dan *recall*, berguna untuk membandingkan model.
    * ***Precision/Recall Trade-off***: Menaikkan *precision* seringkali menurunkan *recall*, dan sebaliknya. Pemilihan *threshold* keputusan sangat memengaruhi trade-off ini.
    * ***Kurva ROC (Receiver Operating Characteristic)***: Plot antara *True Positive Rate* (Recall) melawan *False Positive Rate*. Metrik **Area Under the Curve (AUC)** sering digunakan untuk membandingkan model.

* **Klasifikasi Multikelas, Multilabel, dan Multioutput**:
    * **Multikelas**: Membedakan lebih dari dua kelas yang saling eksklusif (misalnya, angka 0-9). Strategi seperti *One-vs-Rest* (OvR) dan *One-vs-One* (OvO) dibahas.
    * **Analisis Error**: Menganalisis *confusion matrix* untuk memahami jenis kesalahan yang dibuat model dan mencari cara untuk memperbaikinya.
    * **Multilabel**: Setiap instance dapat memiliki beberapa kelas (misalnya, mengenali beberapa wajah dalam satu gambar).
    * **Multioutput**: Generalisasi dari *multilabel* di mana setiap label bisa menjadi multikelas.

### 1. Memuat Dataset MNIST
Pertama, kita muat dataset MNIST. `keras` menyediakan fungsi praktis untuk ini.

```python
import numpy as np
from tensorflow import keras

# Memuat dataset MNIST
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Mengubah gambar 28x28 menjadi vektor 1D berukuran 784
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

# Normalisasi data (opsional, tetapi sering membantu)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
```

### 2. Melatih Pengklasifikasi Biner
Kita akan menyederhanakan masalah dengan mencoba mengidentifikasi satu digit saja, misalnya angka 5. Ini disebut "5-detector".

```python
from sklearn.linear_model import SGDClassifier

# Membuat label target untuk klasifikasi biner (True untuk 5, False untuk lainnya)
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

# Membuat dan melatih model SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

# Menguji pada satu instance
some_digit = X_train[0] # Ini adalah angka 5
print(f"Prediksi untuk digit pertama (seharusnya True): {sgd_clf.predict([some_digit])}")
```

### 3. Mengukur Kinerja (Precision, Recall, Confusion Matrix)
Akurasi bisa menipu. Mari kita gunakan metrik yang lebih baik menggunakan `cross_val_predict` untuk mendapatkan prediksi "bersih" pada setiap data training.

```python
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Mendapatkan prediksi untuk setiap instance di training set
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

# Menghitung confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_train_5, y_train_pred))

# Menghitung precision dan recall
print("\nPrecision:", precision_score(y_train_5, y_train_pred))
print("Recall:", recall_score(y_train_5, y_train_pred))
print("F1 Score:", f1_score(y_train_5, y_train_pred))
```

### 4. Kurva ROC
Kurva ROC adalah alat lain yang bagus untuk mengevaluasi model biner.

```python
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Mendapatkan decision scores alih-alih prediksi
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")

# Menghitung FPR, TPR untuk berbagai threshold
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Garis putus-putus untuk pengklasifikasi acak
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

# Menghitung Area Under the Curve (AUC)
print("ROC AUC Score:", roc_auc_score(y_train_5, y_scores))
```

### 5. Klasifikasi Multikelas dan Analisis Error
Sekarang kita akan melatih model untuk mengklasifikasi semua 10 digit. Scikit-Learn secara otomatis mendeteksi ketika kita mencoba menggunakan algoritma biner untuk tugas multikelas dan menjalankannya dengan strategi OvR (One-vs-Rest).

```python
from sklearn.preprocessing import StandardScaler

# Melatih pada semua 10 kelas (menggunakan y_train, bukan y_train_5)
# Scaling dapat meningkatkan performa
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Mendapatkan prediksi multikelas
y_train_pred_multi = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)

# Membuat confusion matrix multikelas
conf_mx = confusion_matrix(y_train, y_train_pred_multi)
print("Confusion Matrix Multikelas:\n", conf_mx)

# Visualisasi confusion matrix
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

# Fokus pada error: normalisasi confusion matrix berdasarkan jumlah gambar per kelas
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

# Mengisi diagonal dengan nol untuk hanya melihat error
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()
```
Visualisasi matriks error di atas menunjukkan pasangan angka mana yang sering salah diklasifikasikan oleh model (misalnya, angka 3 sering keliru diprediksi sebagai 5, dan sebaliknya).

